# Хакатон AI.Hack Москва.
## Газпром Нефть

https://www.kaggle.com/c/ai-hack-2018-msk-gpn/leaderboard

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
# предопределим типы колонок для экономии памяти
dtypes = {
    'ID': np.int32,
    'CLIENT': np.int32,
    'DATA_TRANS': str,
    'AZS_NUMBER': np.int16,
    'REGION_AZS': np.int16,
    'VID_NP': np.int16,
    'COL_LITR':  np.float32,
    'CENA_CLIENT': np.float32
    }

In [3]:
train = pd.read_csv('datasets/gpn_b2b/train_data.csv', dtype = dtypes)

In [4]:
test = pd.read_csv('datasets/gpn_b2b/test_data.csv', dtype = dtypes)

In [5]:
def month(s): #для обучения сезонности
    return int(s[3:5])

def year(s):
    return int(s[8:10])

def date(s): #для пристегивания фич
    return s[3:10]

In [6]:
%%time

train['month'] = train['DATA_TRANS'].apply(month).astype(float)

test['month'] = test['DATA_TRANS'].apply(month).astype(float)

CPU times: user 25.7 s, sys: 1.01 s, total: 26.7 s
Wall time: 26.6 s


In [7]:
train_sel = train  # catboost не хватало оперативы и использовал только часть, удалено, т.к. lightgbm показал себя на этих данных лучше

In [8]:
%%time

features = pd.read_csv('datasets/gpn_b2b/features_monthly.csv') # сгенерировано с полным тестом в features_monthly.ipynb

trans_count_d = {} # немножко покешируем т.к. вытаскивание из датафрейма медленнее
ltr_mean_d = {}

for index, row in features.iterrows():
    trans_count_d[row['client_date']] = row['trans_count']
    ltr_mean_d[row['client_date']] = row['ltr_mean']

def trans_count(c):
    try:
        return trans_count_d[c]
    except:
        return np.nan

def ltr_mean(c):
    try:
        return ltr_mean_d[c]
    except:
        return np.nan

CPU times: user 49.6 s, sys: 51.2 ms, total: 49.7 s
Wall time: 49.7 s


In [10]:
%%time

train_sel['date'] = train_sel['DATA_TRANS'].apply(date)
train_sel['client_date'] = train_sel['CLIENT'].astype(str) + '_' + train_sel['date'].astype(str)
train_sel['trans_count'] = train_sel['client_date'].apply(trans_count).astype(float)
train_sel['ltr_mean'] = train_sel['client_date'].apply(ltr_mean).astype(float)

test['date'] = test['DATA_TRANS'].apply(date)
test['client_date'] = test['CLIENT'].astype(str) + '_' + test['date'].astype(str)
test['trans_count'] = test['client_date'].apply(trans_count).astype(float)
test['ltr_mean'] = test['client_date'].apply(ltr_mean).astype(float)

CPU times: user 2min, sys: 6.56 s, total: 2min 7s
Wall time: 2min 7s


In [12]:
%%time

# фичи про оценку размера бизнеса
# до 40 000 литров/месяц – массовый;
# от 40 000 до 150 000 литров/месяц – средний;
# от 150 000 литров/месяц – крупный;

train_sel['biz_estimated_ltr'] = train_sel['trans_count'] * train_sel['ltr_mean']
train_sel['biz_mass'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
train_sel['biz_mid'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
train_sel['biz_big'] = train_sel['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)


test['biz_estimated_ltr'] = test['trans_count'] * test['ltr_mean']
test['biz_mass'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x <= 40000 else 0).astype(np.int16)
test['biz_mid'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 40000 and x <= 150000 else 0).astype(np.int16)
test['biz_big'] = test['biz_estimated_ltr'].apply(lambda x: 1 if x > 150000 else 0).astype(np.int16)

CPU times: user 45.7 s, sys: 4.14 s, total: 49.9 s
Wall time: 49.7 s


In [13]:
train_sel.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,trans_all,biz_estimated_ltr,biz_mass,biz_mid,biz_big
2760147,2760147,33792,23.12.2016 12:55:31,3353,18,1,16.65,34.639999,12.0,12.2016,33792_12.2016,6162.0,60.073219,NaN,3.701712e+05,0,0,1
8612596,8612596,42854,02.04.2017 7:52:16,2393,2,1,36.77,33.860001,4.0,04.2017,42854_04.2017,28108.0,48.110050,NaN,1.352277e+06,0,0,1
14830952,14830952,50442,04.07.2017 9:58:13,1335,20,2,30.00,33.430000,7.0,07.2017,50442_07.2017,3597.0,38.097996,NaN,1.370385e+05,0,1,0
3196077,3196077,22450,30.12.2016 8:34:21,3556,19,2,40.00,33.849998,12.0,12.2016,22450_12.2016,357.0,30.638523,NaN,1.093795e+04,1,0,0
3292617,3292617,15335,20.12.2016 7:20:08,3377,18,2,30.00,31.260000,12.0,12.2016,15335_12.2016,2398.0,72.672676,NaN,1.742691e+05,0,0,1


In [14]:
test.sample(5)

,ID,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,CENA_CLIENT,month,date,client_date,trans_count,ltr_mean,trans_all,biz_estimated_ltr,biz_mass,biz_mid,biz_big
200699,200699,31048,04.02.2018 18:27:14,1360,6,1,39.599998,2.0,02.2018,31048_02.2018,53.0,42.152775,NaN,2.234097e+03,1,0,0
527777,527777,49872,11.01.2018 11:30:19,1467,12,2,39.799999,1.0,01.2018,49872_01.2018,7.0,27.686567,NaN,1.938060e+02,1,0,0
3644,3644,34650,14.03.2018 12:02:59,886,4,1,38.599998,3.0,03.2018,34650_03.2018,110.0,109.624512,NaN,1.205870e+04,1,0,0
450523,450523,4540,21.02.2018 17:58:09,904,20,2,38.509998,2.0,02.2018,4540_02.2018,377.0,42.802525,NaN,1.613655e+04,1,0,0
271904,271904,24550,06.01.2018 20:43:41,5087,24,1,37.230000,1.0,01.2018,24550_01.2018,58531.0,35.132069,NaN,2.056315e+06,0,0,1


In [15]:
cols = ['CLIENT','AZS_NUMBER','REGION_AZS','VID_NP','CENA_CLIENT','month','trans_count','biz_estimated_ltr','ltr_mean']

In [16]:
X_train = train_sel[cols]
y = train_sel['COL_LITR']
X_test = test[cols]

del train, train_sel, test
gc.collect() 

451

In [17]:
import lightgbm

In [51]:
%%time

ctb = lightgbm.LGBMRegressor(n_estimators=48)
ctb.fit(X_train, y)

CPU times: user 6min 6s, sys: 8.82 s, total: 6min 15s
Wall time: 1min 58s


In [52]:
ctb

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=48,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [53]:
pd.DataFrame(X_train.columns, ctb.feature_importances_)

,0
49,CLIENT
376,AZS_NUMBER
318,REGION_AZS
96,VID_NP
102,CENA_CLIENT
7,month
74,trans_count
108,biz_estimated_ltr
310,ltr_mean
0,trans_all


In [54]:
y_test = ctb.predict(X_test)

In [55]:
y_test.shape

(1000000,)

In [56]:
X_test.shape

(1000000, 10)

In [57]:
sample_submission = pd.read_csv('datasets/gpn_b2b/sample_submission.csv')

In [58]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,0.0
999996,999996,0.0
999997,999997,0.0
999998,999998,0.0
999999,999999,0.0


In [59]:
sample_submission['COL_LITR'] = y_test

In [60]:
sample_submission.tail()

,ID,COL_LITR
999995,999995,152.915393
999996,999996,168.871750
999997,999997,339.140611
999998,999998,40.482968
999999,999999,34.753451


In [61]:
sample_submission.to_csv('gpn_b2b_allm.csv.gz', compression='gzip',  index=False)

In [ ]:
# estimators 128 и 64